# Hyperparameter tuning for fine-tuned Chronos-2 models

Questo notebook avvia l'iper-tuning partendo da checkpoint **già finetunati**.
I checkpoint si trovano nella cartella `outputs/chronos2_sft` (generale e per categorie).


In [ ]:
# Example: create a JSON mapping for category models
import json
from pathlib import Path

category_models = {
    'Information Technology': 'outputs/chronos2_sft/categories/Information_Technology/finetuned-ckpt',
    'Health Care': 'outputs/chronos2_sft/categories/Health_Care/finetuned-ckpt',
}
Path('category_models.json').write_text(json.dumps(category_models, indent=2), encoding='utf-8')
print('Wrote category_models.json')

# Run tuning for the general model and category models
!python ../core/tuning.py \
    --tune-general \
    --tune-categories \
    --general-model-path outputs/chronos2_sft/finetuned-ckpt \
    --category-models-json category_models.json \
    --output-dir outputs/hyperparameter_tuning
